# Examples of `snews_pt` usage

Last updated: 11/12/2023 <br>
This notebook contains the most basic publication and subscription examples.

## Subscription

In [1]:
from snews_pt.snews_sub import Subscriber

Subscriber().subscribe()

You are subscribing to ALERT
Broker:kafka://kafka.scimma.org/snews.alert-firedrill
Done


---
# Publication

Here we create a message to which we pass the following arguments; <br>
`detector_name`, `machine_time`, `neutrino_time`, `p_val`, `p_values`,`t_bin_width` <br>
As described in [Message Schema Section](https://snews-publishing-tools.readthedocs.io/en/latest/user/command_line_interface.html) of the documentation. The tiers are decided automatically by the scripts based on the input arguments passed. <br>

Here the argument `neutrino_time` indicates that the message should be sent to "Coincidence Tier" for coincidence checks with the other experiments. <br>

Similarly, the input arguments; `p_values` and `t_bin_width` indicates that the message should go to "Significance Tier" for combined significance computations. For more please refer to the documentations.

## Old way

In [1]:
# from snews_pt.snews_pub import SNEWSTiersPublisher
# from datetime import  datetime

# test_time = datetime.utcnow().isoformat()
# message = SNEWSTiersPublisher(detector_name='LZ', machine_time=test_time, neutrino_time=test_time, p_val=0.0007, p_values=[0.0007,0.0008,0.0009],t_bin_width=0.07)
# # print(message.message_data)
# # print(message.tiernames)
# # print('Sending message')
# message.send_to_snews()

## New construction

In [2]:
from snews_pt import messages
# import importlib
# importlib.reload(messages)
from datetime import datetime

test_time = datetime.utcnow().isoformat()

## Valid messages

**1) message from a valid detector with valid neutrino times and valid p values.**
This creates two messages; CoincidenceTierMessage and SignificanceTierMessage

In [3]:
msg = messages.SNEWSMessageBuilder(detector_name='LZ', 
                                   machine_time=test_time, 
                                   neutrino_time=test_time, 
                                   p_val="0.000", 
                                   p_values=[0.0007,0.0008,0.0009],
                                   t_bin_width=0.07, 
                                   is_test=True,)

msg.messages

[SNEWSCoincidenceTierMessage
 ---------------------------
             _id : LZ_CoincidenceTier_2023-12-11T15:53:19.155054
  schema_version : 1.3.0
   detector_name : LZ
    machine_time : 2023-12-11T15:53:19.155054
   neutrino_time : 2023-12-11T15:53:19.155054
           p_val : 0.000
         is_test : True
        p_values : [0.0007, 0.0008, 0.0009]
     t_bin_width : 0.07
 ,
 SNEWSSignificanceTierMessage
 ----------------------------
             _id : LZ_SignificanceTier_2023-12-11T15:53:19.155054
  schema_version : 1.3.0
   detector_name : LZ
    machine_time : 2023-12-11T15:53:19.155054
        p_values : [0.0007, 0.0008, 0.0009]
     t_bin_width : 0.07
         is_test : True
   neutrino_time : 2023-12-11T15:53:19.155054
           p_val : 0.000
 ]

Above the color code refers to the <br>
red:"base", (id, version, name) <br>
blue:"required", (tier specific) <br>
black:"accepted" (machine time, p value, retraction reason etc. known-nonrequired ones) and <br>
green:"meta" field arguments.

In [4]:
# msg.messages[0].fields, 
msg.messages[0].basefields, msg.messages[0].reqfields, msg.messages[0].meta.keys()

(['_id', 'schema_version', 'detector_name'],
 ['neutrino_time'],
 dict_keys(['p_values', 't_bin_width']))

**2) Similarly, as long as the neutrino time iso-formattable** <br>
Notice the neutrino time comes from the past but it is a test, and therefore ignored

In [5]:
msg = messages.SNEWSMessageBuilder(detector_name='LZ', 
                          machine_time=test_time, 
                          neutrino_time="2023-06-12 18:30", 
                          is_test=True)

msg.messages

[SNEWSCoincidenceTierMessage
 ---------------------------
             _id : LZ_CoincidenceTier_2023-12-11T15:53:19.155054
  schema_version : 1.3.0
   detector_name : LZ
    machine_time : 2023-12-11T15:53:19.155054
   neutrino_time : 2023-06-12T18:30:00
           p_val : None
         is_test : True
 ]

## Invalid Messages

**1) With no valid argument**

In [6]:
msg = messages.SNEWSMessageBuilder(test=1)
msg

SNEWSMessageBuilder:
No messages have been built.

**2) unknown detector name**

In [ ]:
msg = messages.SNEWSMessageBuilder(detector_name='aseyhsreshdshdshsdgsdg', 
                                  machine_time=test_time, 
                                  neutrino_time=test_time, 
                                  p_val=0.0007, 
                                  p_values=[0.0007,0.0008,0.0009],
                                  t_bin_width=0.07, 
                                  is_test=True)

**3) Neutrino time in the past and it is not a test message.**

In [ ]:
msg = messages.SNEWSMessageBuilder(detector_name='LZ', 
                                  machine_time=test_time, 
                                  neutrino_time="2023-06-12 18:30",  
                                  is_test=False)

**4) p values beyond 0 and 1**

In [ ]:
msg = messages.SNEWSMessageBuilder(detector_name='LZ', 
                                   machine_time=test_time, 
                                   p_values=[-0.0007,0.0008,0.0009],
                                   t_bin_width=0.07, 
                                   is_test=False)

## Other attributes and functions

In [8]:
msg = messages.SNEWSMessageBuilder(detector_name='LZ', 
                                  machine_time=test_time, 
                                  neutrino_time=test_time, 
                                  p_val="0.000", 
                                  p_values=[0.0007,0.0008,0.0009],
                                  t_bin_width=0.07, 
                                  is_test=True,)

In [9]:
msg.messages[0]

SNEWSCoincidenceTierMessage
---------------------------
            _id : LZ_CoincidenceTier_2023-12-11T15:53:19.155054
 schema_version : 1.3.0
  detector_name : LZ
   machine_time : 2023-12-11T15:53:19.155054
  neutrino_time : 2023-12-11T15:53:19.155054
          p_val : 0.000
        is_test : True
       p_values : [0.0007, 0.0008, 0.0009]
    t_bin_width : 0.07


In [9]:
msg.send_messages()

----------------------------------------------------------------
Sending message to CoincidenceTier on kafka://kafka.scimma.org/snews.experiments-firedrill
_id                :LZ_CoincidenceTier_2023-12-11T15:45:15.394059
schema_version     :1.3.0
detector_name      :LZ
machine_time       :2023-12-11T15:45:15.394059
neutrino_time      :2023-12-11T15:45:15.394059
p_val              :0.000
is_test            :True
meta               :
sent_time          :2023-12-11T15:45:45.677277
----------------------------------------------------------------
Sending message to SignificanceTier on kafka://kafka.scimma.org/snews.experiments-firedrill
_id                :LZ_SignificanceTier_2023-12-11T15:45:15.394059
schema_version     :1.3.0
detector_name      :LZ
machine_time       :2023-12-11T15:45:15.394059
p_values           :[0.0007, 0.0008, 0.0009]
t_bin_width        :0.07
is_test            :True
meta               :
sent_time          :2023-12-11T15:45:45.679676


Notice below, the fields that do not belong to the tier in question are initially appended as "meta" fields. <br>
However, when sending the messages to SNEWS, the program checks this and avoids redundancies by only keeping the "meta" fields, if the keys are not used in any other valid message.

In [10]:
msg.messages[0].message_data, msg.messages[0].meta

({'_id': 'LZ_CoincidenceTier_2023-12-11T15:53:19.155054',
  'schema_version': '1.3.0',
  'detector_name': 'LZ',
  'machine_time': '2023-12-11T15:53:19.155054',
  'neutrino_time': '2023-12-11T15:53:19.155054',
  'p_val': '0.000',
  'is_test': True},
 {'p_values': [0.0007, 0.0008, 0.0009], 't_bin_width': 0.07})

In [11]:
msg.messages[1].message_data, msg.messages[1].meta

({'_id': 'LZ_SignificanceTier_2023-12-11T15:53:19.155054',
  'schema_version': '1.3.0',
  'detector_name': 'LZ',
  'machine_time': '2023-12-11T15:53:19.155054',
  'p_values': [0.0007, 0.0008, 0.0009],
  't_bin_width': 0.07,
  'is_test': True},
 {'neutrino_time': '2023-12-11T15:53:19.155054', 'p_val': '0.000'})

### See the message schemas.

In [12]:
msg.messages[0].print_schema()

Message schema for SNEWSCoincidenceTierMessage
_id                  : (SET AUTOMATICALLY)
schema_version       : (SET AUTOMATICALLY)
detector_name        : (SET AUTOMATICALLY)
neutrino_time        : (REQUIRED USER INPUT)
machine_time         : (USER INPUT)
p_val                : (USER INPUT)
is_test              : (USER INPUT)


In [13]:
msg.messages[1].print_schema()

Message schema for SNEWSSignificanceTierMessage
_id                  : (SET AUTOMATICALLY)
schema_version       : (SET AUTOMATICALLY)
detector_name        : (SET AUTOMATICALLY)
p_values             : (REQUIRED USER INPUT)
t_bin_width          : (REQUIRED USER INPUT)
machine_time         : (USER INPUT)
is_test              : (USER INPUT)
